In [1]:
import tensorflow as tf

In [8]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf
import numpy as np

FLAGS = None

In [3]:
# Import data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [54]:
W = tf.Variable(tf.zeros([784, 10]))
print(W)
print(b)

print(F)

<tf.Variable 'Variable_78:0' shape=(784, 10) dtype=float32_ref>
<tf.Variable 'Variable_71:0' shape=(10,) dtype=float32_ref>
Tensor("Reshape_39:0", shape=(7840, 1), dtype=float32)


In [163]:
# Create the model
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(x, W) + b

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(
  tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

grad_W, grad_b = tf.gradients(xs=[W, b], ys=cross_entropy)

# Gradient Step
learning_rate = 0.1
new_W = W.assign(W - learning_rate * grad_W)
new_b = b.assign(b - learning_rate * grad_b)

#w_flat = tf.contrib.layers.flatten(W)
# b_flat = tf.contrib.layers.flatten(b)

w_flat = tf.reshape(W, [7840,])
flout = tf.concat([w_flat,b],0)
#f = tf.concat([w_flat, b], 0)
hess = tf.hessians(cross_entropy, [b])

#I would expect this to work in giving us our Hessian but it does not...

print(b)
print(w_flat)
print(flout)

##???
##hessfull = tf.hessians(cross_entropy, [flout])


#copy paste from Matthews script
def jacobian(y_flat, x):
    n = y_flat.shape[0]
    #
    loop_vars = [
        tf.constant(0, tf.int32),
        tf.TensorArray(tf.float32, size=n),
    ]
    #
    _, jacobian = tf.while_loop(
        lambda j, _: j < n,
        lambda j, result: (j+1, result.write(j, jacobian_piece(y_flat, x, j))),
        loop_vars,swap_memory=True)
    #
    return jacobian.stack()

def hessian_via_jacobian(loss, parameters):
  #
  #
  ### Note: We can call tf.trainable_variables to get GraphKeys.TRAINABLE_VARIABLES 
  ### because we are using g as our default graph inside the "with" scope. 
  # Get trainable variables
  # Get gradients of loss with repect to parameters
  dloss_dw = tf.gradients(loss, parameters)[0]
  #
  hess = jacobian(dloss_dw, parameters)
  return hess


def jacobian_piece(vect, parameters, index):
  dfx_i = tf.slice(vect, begin=[index,0] , size=[1,1])
  ddfx_i = tf.gradients(dfx_i, parameters)[0] 
  return ddfx_i
#END COPY PAST



sess = tf.InteractiveSession()
tf.global_variables_initializer().run()



#inv_hess = [tf.matrix_inverse(h) for h in hessy]

# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                  y_: mnist.test.labels}))

# Train
for _ in range(1):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run([new_W, new_b], feed_dict={x: batch_xs, y_: batch_ys})


    #Computing the eigenvectors and eigenvalues of only the bias's works fine
    check = tf.self_adjoint_eig(hess)
    
    #Why does this have a funny shape and not eigendecompose?
    u = hessian_via_jacobian(cross_entropy, [W, b])
    #print(u)

    print(sess.run([tf.reduce_sum(tf.abs(hessian_via_jacobian(cross_entropy, [W, b])))], feed_dict={x: batch_xs, y_: batch_ys}))
    print(sess.run([u], feed_dict={x: batch_xs, y_: batch_ys}))
    
    #Why does this not decompose?
    #print(sess.run([tf.self_adjoint_eig(u)], feed_dict={x: batch_xs, y_: batch_ys}))
    
# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                  y_: mnist.test.labels}))

<tf.Variable 'Variable_268:0' shape=(10,) dtype=float32_ref>
Tensor("Reshape_117:0", shape=(7840,), dtype=float32)
Tensor("concat_20:0", shape=(7850,), dtype=float32)
0.098
[2211.656]
[array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
    

ValueError: Dimensions must be equal, but are 784 and 10 for 'SelfAdjointEigV2_179' (op: 'SelfAdjointEigV2') with input shapes: [?,784,10].